# Polarización de comentarios

## Objetivo

Aplicación de las herramientas de análisis de texto de Python en los comentarios de Facebook de la Universidad del Sinú - Elías Bechara Zainúm

### Generalidades

* Através de Power Bi, se accede a la API de QUINTLY para obtener los comentarios de la cuenta de Facebook del perfil instirucional
* La salida de la solicitud anterior es un JSON que puede ser tratado por Power Bi, hasta convertirlo en una tabla
* En Power Bi se puedem ejecutar scripts de Python, sin embargo en el presente documento se definirá el algoritmo para analizar los comentarios de facebook desde un excel
* La idea de este documento es que posteriormente se pueda aplicar esta herramienta directamente en Power Bi

#### Solicitud a la API

La siguiente es la solicitud que se realiza a la API de Quintly, desde Power Bi.
La conexión es de tipo Basic y se deben ingresar el usuario y contraseña de la API.

https://api.quintly.com/v0.9/qql?qqlQuery=SELECT%20nameWithAppendix%20AS%20pageName,%20facebookComments.time%20AS%20createTime,%20importTime,%20facebookComments.message%20AS%20message,%20facebookComments.link%20AS%20link,%20facebookComments.attachmentPreviewPicture,%20attachmentType,%20userName,%20facebookOwnPosts.message%20AS%20originalPost%20FROM%20facebookComments%20INNER%20JOIN%20profiles%20ON%20facebookComments.profileId=profiles.id%20JOIN%20facebookOwnPosts%20ON%20facebookComments.externalPostId=facebookOwnPosts.externalId%20WHERE%20facebookComments.time%20>=%20facebookOwnPosts.time%20ORDER%20BY%20createTime&startTime=2021-02-01&endTime=2021-04-01&interval=weekly&profileIds=404998

In [41]:
import csv
import emoji
import time

In [42]:
coments_facebook=open('Tabla_de_comentarios.csv','r')
coments_facebook_r=csv.reader(coments_facebook)
def deEmojify(text):
    return emoji.get_emoji_regexp().sub(r'', text.decode('utf8'))

with open('Texto_comentarios.csv','w', newline='',encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["success","pageName","createTime","importTime","message","link","attachmentPreviewPicture","attachmentType","userName","originalPost"])
    
    with open('Tabla_de_comentarios_v2.csv', newline='',encoding='utf-8') as File1:  
        reader = csv.reader(File1)
        for row in reader:
            text_row=row[4]
            #print(text_row)
            allchars= [str for str in text_row]
            emoji_list=emoji.emoji_lis(text_row)
            aux2=[]
            if(len(emoji_list)>0):
                aux1=emoji_list
                for x in aux1:
                    aux2.append(x['emoji'])
                #print(aux2)
            clean_text=' '.join([str for str in text_row.split() if not any(i in str for i in aux2)])
            data=[row[0],row[1],row[2],row[3],clean_text,row[5],row[6],row[7],row[8],row[9]]
            #print(data)
            #print(allchars)
            writer.writerow(data)

In [43]:
import pandas as pd
data=pd.read_csv("Texto_comentarios.csv")
comentaries=data['message']
comentaries[0]

'data.message'

In [44]:
import textblob

In [45]:
import requests

url = "https://google-translate1.p.rapidapi.com/language/translate/v2"

payload = "q=Hello%2C%20world!&source=en&target=es"
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'accept-encoding': "application/gzip",
    'x-rapidapi-key': "######", #Aqui va la api key
    'x-rapidapi-host': "google-translate1.p.rapidapi.com"
    }

In [46]:
comentaries_list=[]
popularity_list=[]
subjectivity_list=[]
data = data.dropna(subset=["message"])
#data2=data.drop(0)
data2=data
for coment in data2['message']:
    print(coment)
    try:
        #SE TRADUCE EL COMENTARIO HACIENDO UNA PETICION A LA API
        payload = "q="+coment+"&source=es&target=en"
        response = requests.request("POST", url, data=payload, headers=headers)
        texto=response.json()
        en_coment=texto["data"]["translations"][0]["translatedText"]
    except:
        en_coment=coment
    comentaries_list=coment
    print(en_coment)
    analysis=textblob.TextBlob(en_coment)
    analysis= analysis.sentiment
    print(analysis)
    popularity=analysis.polarity
    popularity_list.append(popularity)
    subjectivity=analysis.subjectivity
    subjectivity_list.append(subjectivity)
    time.sleep(1)

data.message
data.message
Sentiment(polarity=0.0, subjectivity=0.0)
Excelente
Excellent
Sentiment(polarity=1.0, subjectivity=1.0)
Buenas tardes
Good afternoon
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Buenas tardes
Good afternoon
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Buenas tardes
Good afternoon
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Buenas tardes
Good afternoon
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Buenas tardes.
Good afternoon.
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Buenas tardes
Good afternoon
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Exelente
Excellent
Sentiment(polarity=1.0, subjectivity=1.0)
Buenas tardes
Good afternoon
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Buenas tardes a todos
Good afternoon everyone
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
.
.
Sentiment(polarity=0.0, subjectivity=0.0)
Buenas tardes
Good afternoon
Sentiment(polarity=0.7, sub

Exelente
Exelente
Sentiment(polarity=0.0, subjectivity=0.0)
Hermosos poemas
Hermosos poemas
Sentiment(polarity=0.0, subjectivity=0.0)
Aaron Parodi Quiroga
Aaron Parodi Quiroga
Sentiment(polarity=0.0, subjectivity=0.0)
Julio Sierra Domínguez
Julio Sierra Domínguez
Sentiment(polarity=0.0, subjectivity=0.0)
gracias por compartir su hermoso poema
gracias por compartir su hermoso poema
Sentiment(polarity=0.0, subjectivity=0.0)
Saludos a mi comadre Irene Tapias. Un abrazo.
Saludos a mi comadre Irene Tapias. Un abrazo.
Sentiment(polarity=0.0, subjectivity=0.0)
Maravillosa antología. Gracias Universidad del Sinú. Gracias
Maravillosa antología. Gracias Universidad del Sinú. Gracias
Sentiment(polarity=0.0, subjectivity=0.0)
Abrazos para gestores y participantes.
Abrazos para gestores y participantes.
Sentiment(polarity=0.0, subjectivity=0.0)
Buenas tardes profe
Buenas tardes profe
Sentiment(polarity=0.0, subjectivity=0.0)
Irene Tapias
Irene Tapias
Sentiment(polarity=0.0, subjectivity=0.0)
Muchas

KeyboardInterrupt: 

In [47]:
print(popularity_list)

[0.0, 1.0, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 1.0, 0.7, 0.7, 0.0, 0.7, 0.7, 0.7, 0.7, 0.7, 0.43, 0.0, 0.7, 0.0, 0.7, 0.7, 0.0, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
with open('Texto_comentarios_final.csv','w', newline='',encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["success","pageName","createTime","importTime","message","link","attachmentPreviewPicture","attachmentType","userName","originalPost","English_Coment","polaridad","subjetividad"])
    
    with open('Texto_comentarios.csv', newline='',encoding='utf-8') as File1:  
        reader = csv.reader(File1)
        int i=0
        for row in reader:
            data=[row[0],row[1],row[2],row[3],clean_text,row[5],row[6],row[7],row[8],row[9],comentaries_list[i],popularity_list[i],subjectivity_list[i]]
            writer.writerow(data)
            i=i+1

In [10]:
import requests

url = "https://google-translate1.p.rapidapi.com/language/translate/v2"

payload = "q=Hello%2C%20world!&source=en&target=es"
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'accept-encoding': "application/gzip",
    'x-rapidapi-key': "###",#Aquí va la API KEY
    'x-rapidapi-host': "google-translate1.p.rapidapi.com"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

{"data":{"translations":[{"translatedText":"¡Hola Mundo!"}]}}


In [36]:
texto=response.json()
print(texto["data"]["translations"][0]["translatedText"])

¡Hola Mundo!


In [40]:
payload = "q=Me gusta esa canción&source=es&target=en"
response = requests.request("POST", url, data=payload, headers=headers)
texto=response.json()
import time
time.sleep(15)
print(texto["data"]["translations"][0]["translatedText"])

I like that song
